In [11]:
# Automated stock picking based on following logic:
# Calculate the average PE ratio of all the stocks in the list.
# Filter stocks that have a PE ratio lower than the average PE ratio.
# Further filter stocks with a momentum health indicator (Percentage of positive days - percentage of negative days greater than 10).
# Sort the remaining stocks by their 2024 return in descending order.
# Select the top 10 stocks.


import numpy as np
import yfinance as yf
import pandas as pd

def get_sp500_tickers():
    table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    tickers = table[0]['Symbol'].tolist()
    return tickers

def fetch_and_analyze_data(ticker):
    try:
        adjusted_ticker = ticker.replace('.', '-')
        stock_data = yf.Ticker(adjusted_ticker).history(start='2023-12-29', end='2024-12-31')

        if not stock_data.empty:
            stock_data.index = stock_data.index.tz_localize(None)
            
            start_price = stock_data.iloc[0]['Close']
            end_price = stock_data.iloc[-1]['Close']
            annual_return = (end_price - start_price) / start_price * 100
            
            stock_data['Daily Return'] = stock_data['Close'].pct_change()
            std_dev = stock_data['Daily Return'].std() * (252**0.5)
            sharpe_ratio = (annual_return / 100 - risk_free_rate) / std_dev
            
            # Fetch P/E ratio using yfinance
            pe_ratio = yf.Ticker(adjusted_ticker).info.get('trailingPE', None)

            # Calculate Momentum Health Indicator
            stock_data['up_days'] = stock_data['Daily Return'] > 0
            stock_data['down_days'] = stock_data['Daily Return'] < 0

            up_days_percentage = stock_data['up_days'].sum() / len(stock_data) * 100
            down_days_percentage = stock_data['down_days'].sum() / len(stock_data) * 100

            stock_data['Momentum Health Indicator'] = up_days_percentage - down_days_percentage

            # Drop the temporary columns
            stock_data.drop(columns=['up_days', 'down_days'], inplace=True)
            
            # Calculate Delta-Normal VAR at 95% confidence level
            expected_return = stock_data['Daily Return'].mean()
            var_95 = expected_return - 1.96 * std_dev
            stock_data['VAR'] = var_95

            return {'Ticker': ticker, '2024 Return': annual_return, '2024 Std Dev': std_dev, 'Sharpe Ratio': sharpe_ratio, 'P/E Ratio': pe_ratio, 'Momentum Health Indicator': stock_data['Momentum Health Indicator'].iloc[-1], 'VAR': stock_data['VAR'].iloc[-1]}, stock_data
        else:
            return None, None
    except Exception as e:
        print(f"Skipping {ticker} due to error: {e}")
        return None, None

sp500_tickers = get_sp500_tickers()
returns_data = []
risk_free_rate = 0.02

writer = pd.ExcelWriter('sp500_data.xlsx', engine='openpyxl')

for ticker in sp500_tickers:
    result, stock_data = fetch_and_analyze_data(ticker)
    if result and stock_data is not None:
        returns_data.append(result)
        stock_data.to_excel(writer, sheet_name=result['Ticker'])

returns_df = pd.DataFrame(returns_data)

# Convert P/E Ratio to numeric, forcing errors to NaN
returns_df['P/E Ratio'] = pd.to_numeric(returns_df['P/E Ratio'], errors='coerce')

# Drop rows with NaN P/E Ratio
returns_df = returns_df.dropna(subset=['P/E Ratio'])

# Filter out extremely high P/E Ratios that might skew the average
returns_df = returns_df[returns_df['P/E Ratio'] < 1000]

average_pe_ratio = returns_df['P/E Ratio'].mean()

print("Average PE Ratio: ", average_pe_ratio, "\n")


filtered_df = returns_df[(returns_df['P/E Ratio'] < average_pe_ratio) & (returns_df['Momentum Health Indicator'] > 10)]
filtered_df_sorted = filtered_df.sort_values(by='2024 Return', ascending=False)
top_10_stocks = filtered_df_sorted.head(10)
top_10_stocks.to_excel(writer, sheet_name='Top 10 Stocks', index=False)

writer.close()

print(top_10_stocks[['Ticker', '2024 Return', '2024 Std Dev', 'Sharpe Ratio', 'P/E Ratio', 'Momentum Health Indicator', 'VAR']])


Average PE Ratio:  35.701178391614256 

    Ticker  2024 Return  2024 Std Dev  Sharpe Ratio  P/E Ratio  \
476    VST   267.350480      0.567212      4.678151  30.356602   
436   TRGP   109.253574      0.240299      4.463339  35.370705   
220   GDDY    87.876780      0.233776      3.673457  15.449438   
38     APO    81.345737      0.319786      2.481210  16.261099   
203    FOX    69.504967      0.204744      3.297037  11.251589   
202   FOXA    67.198073      0.210526      3.096912  11.809290   
208     GE    66.120081      0.305103      2.101587  33.760870   
493    WMB    62.469501      0.195985      3.085407  23.580511   
26     AXP    60.611815      0.241544      2.426549  21.757732   
449     TT    53.284587      0.233322      2.198016  34.486240   

     Momentum Health Indicator       VAR  
476                  18.650794 -1.105908  
436                  22.222222 -0.467926  
220                  22.619048 -0.455579  
38                   13.888889 -0.624205  
203               